# IN2110 Oblig 2a
## Oppgave 2 og 3
Etter at du har trent modellen fra kommandolinja, gjøres resten av programmeringen, rapporteringen og besvarelse ellers i dette dokumentet. Lag celler for markdown og kode etter behov.

In [2]:
import spacy
from helpers.conllu import ConlluDoc
print("spaCy version:",spacy.__version__) 

spaCy version: 3.7.2


In [133]:
nb = spacy.load("output/model-best")

dev = ConlluDoc.from_file("no_bokmaal-ud-dev.conllu")
dev_spacy = dev.to_spacy(nb)
dev_spacy_unlabelled = dev.to_spacy(nb, keep_labels=False)

# Kjører print-test for å kontrollere at setninger parsert av modellen vår er merka med dependensrelasjoner
doc = dev_spacy_unlabelled[13]
print(doc)
print(doc.has_annotation("DEP"))
doc_parsed = nb(doc)
print(doc_parsed.has_annotation("DEP"))

I rene ord. 
False
True


In [134]:
def parse(docs):

    # trenger jeg å utdype denne koden nor særlig?
    if not isinstance(docs, list):
        raise TypeError("Input argument 'docs' must be of instance 'list'")

    docs_parsed = [nb(doc) for doc in docs]

    return docs_parsed

In [129]:
def attachment_score(true, pred):
    """Tar inn to lister av doc der liste nr 2 har predikert depedens. Finner andel korrekte head (UAS), og korrekte head+dependensrelasjoner (LAS)
    Merk at depedensrelasjonene som SpaCy estimerer har "ROOT" med store bokstaver, mens våre data har "root" med små.
    Dette er den samme relasjonen, og skal teller som korrekt estimert """
    #TODO

    for i in [true, pred]:
        if not isinstance(i, list):
            raise TypeError(f"Inputs 'true' and 'pred' must be of type list. Got type {type(i)}")
    if not len(true) == len(pred):
        raise LengthException(f"len(true) != len(pred). len(true): {len(true)}, len(pred): {len(pred)}")

    correct_head_count = 0
    correct_deprel_count = 0
    word_count = 0

    for i in range(len(true)):    
        for doc_true, doc_pred in zip(true[i], pred[i]):
            word_count += 1
            # denne if-testen sjekker om indeksen til det sanne hodet er den samme som indeksen til det predikerte hodet
            if doc_true.head.i == doc_pred.head.i:
                correct_head_count += 1
                # ny if-test sjekker om den sanne dependensrelasjonen tilsvarer den predikerte med små bokstaver (siden vår modell predikerer 'ROOT'
                # i motsetning til 'root'). Det er denne if-testen som utgjør grunnlaget for LAS-skåren.
                if doc_true.dep_ == doc_pred.dep_.lower():
                    correct_deprel_count += 1

    uas = correct_head_count / word_count
    las = correct_deprel_count / word_count

    # Prosedyren returnerer en ordbok som inneholder all nyttig informasjon om parseringa.
    return {"wordcount": word_count, "correct head count": correct_head_count, "uas": uas, "correct deprel count": correct_deprel_count, "las": las}

In [138]:
import re

"""lager en prosedyre som kjører en trebank gjennom parseren fra tidligere, lager en dataframe av ordboka denne har som output, og leger til en
egen kolonne i denne med navnet på språket som blei parsert. Språknavnet hentes ut fra filnavnet ved hjelp av regex. Deretter lager jeg en samla
dataframe med resultatene fra bokmål, nynorsk og nynorsk talespråk."""
def read_conllu(file):
    dev = ConlluDoc.from_file(file)
    dev, dev_unlabelled = dev.to_spacy(nb), dev.to_spacy(nb, keep_labels=False)
    length = dev
    
    true = list(dev)
    pred = parse(dev_unlabelled)

    df = pd.DataFrame(attachment_score(true, pred), index=[0])
    spraknavn = re.findall(r'_(\w+)-', file)
    df.insert(0, "language", spraknavn)
      
    return df

df = pd.concat([read_conllu("no_bokmaal-ud-dev.conllu"), read_conllu("no_nynorsk-ud-dev.conllu"), read_conllu("no_nynorsklia-ud-dev.conllu")], ignore_index=True)
df

,lang,wordcount,correct head count,uas,correct deprel count,las
0,bokmaal,36369,31966,0.878935,30819,0.847398
1,nynorsk,31250,21047,0.673504,18290,0.585280
2,nynorsklia,10163,4547,0.447407,3536,0.347929


### Kommentarer til resultatene ###

Som forventa er UAS-skåren høyere enn LAS-skåren. Dette er fordi UAS ser bort ifra dependensrelasjon og kun vurderer om riktig hode er blitt predikert. LAS krever på sin side riktig deåpendensrelasjon i tillegg, for å karakterisere prediskjonen som korrekt. Det viser seg videre at modellen vår presterer betydelig bedre på bokmål enn på skriftlig nynorsk og nynorsk muntlig tale. Det forteller oss at bokmål og nynorsk er såpass forskjellige at en bokmålstrent parserer ikke egnes til å parsere nynorsk - aller minst nynorsk talespråk, som her får dårligst skår av alle.